In [2]:
# importing module
from pandas import *
from statistics import mean
import numpy as np
from helper760 import read_inputs
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler 
import tensorflow as tf
from sklearn.model_selection import KFold
n_split=8           # define number for k
re_train_1 = True  # define for the step 1 (CT+Clinic -> Death Day), do we want to retrain the model?
re_train_2 = False  # define for the step 2 (CT -> Death Day), do we want to retrain the model?


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  2


# Step 1 - Use both clinic and ct data to predict death day
Note that we have only around 549 real samples with known death day for us to train the model

Currently, we use these 549 samples as training set, and the rest will be the prediction set


In [3]:
# read data for step 1
Clininc_Data,Outcome_Data,CT_Data = read_inputs()

In [4]:
# figure out who has known death day
n_sample = len(Outcome_Data[0])
fill_train_idx = []
fill_test_idx = []

for i in range(n_sample):
    if Outcome_Data[0][i]!=0:
        fill_train_idx.append(i)
    else:
        fill_test_idx.append(i)

In [5]:
# prepare the data for training
# in this case we use both the clinic data and ct data
X1 = np.array(CT_Data)
X1 = X1.T
X1 = X1.astype(float)
print(X1.shape)

X2 = np.array(Clininc_Data)
X2 = X2.T
X2 = X2[:,4:14]
X2 = X2.astype(float)
print(X2.shape)

X = np.hstack((X1,X2))
print(X.shape)

X_fill_train = X[fill_train_idx,:]
print(X_fill_train.shape)

X_fill_test = X[fill_test_idx,:]
print(X_fill_test.shape)

y_fill_train = np.array(Outcome_Data[0])
y_fill_train = y_fill_train.astype(int)
y_fill_train = np.asmatrix(y_fill_train)
y_fill_train = y_fill_train.T
y_fill_train = y_fill_train[fill_train_idx,:]
print(y_fill_train.shape)


(9223, 11)
(9223, 10)
(9223, 21)
(549, 21)
(8674, 21)
(549, 1)


In [6]:
# train the model use the 549 samples, with clinic data + ct data
if re_train_1 == True:
    models = []
    evas = []
    counter = 0
    for train_index,test_index in KFold(n_split).split(X_fill_train):
        print("training fold "+str(counter)+" total "+str(n_split) +" folds")

        x_train,x_test = X_fill_train[train_index], X_fill_train[test_index]
        y_train,y_test = y_fill_train[train_index], y_fill_train[test_index]
    
        # define the keras model
        model = Sequential()
        model.add(Dense(32, input_dim=21, use_bias=True))
        model.add(Dense(64, activation='relu',use_bias=True))
        model.add(Dense(128, activation='relu',use_bias=True))
        model.add(Dense(256, activation='relu',use_bias=True))
        model.add(Dense(512, activation='relu',use_bias=True))
        model.add(Dense(1))
        opt = keras.optimizers.Adam(learning_rate=0.0005)
        # compile the keras model
        model.compile(loss='MeanSquaredError', optimizer=opt, metrics=['MeanSquaredError'])
        # fit the keras model on the dataset
        model.fit(x_train, y_train, epochs=3000, batch_size=5,verbose=1)
        eva = model.evaluate(x_test,y_test)
        
        print("fold "+str(counter)+" eva: "+str(eva[0]))
        counter = counter + 1
        
        models.append(model)
        evas.append(eva)
    for i in range(len(evas)):
        evas[i] = evas[i][0]
    # find the best evaluated models
    model = models[evas.index(min(evas))]

else:
    model = tf.keras.models.load_model("NN_models/keras_model_step_1", custom_objects=None, compile=True, options=None)


training fold 0 total 8 folds


2022-05-09 18:39:45.108831: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-09 18:39:45.229218: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 18:39:45.229403: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 18:39:45.229563: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

KeyboardInterrupt: 

In [ ]:
# let's see how the model fits the original training data
y_fill_train_predict = model.predict(X_fill_train)
err_sum = 0
for i in range((y_fill_train_predict.shape)[0]):
    err_sum = err_sum + (abs(y_fill_train_predict[i,0]-y_fill_train[i,0]))
print("average death day error across all 549 traing sample: "+str(err_sum/(y_fill_train_predict.shape)[0]))

average death day error across all 549 traing sample: 420.6206548027219


In [ ]:
# save the model used for step 1
from datetime import datetime
if re_train_1 == True:
    model.save("NN_models/keras_model_step_1")


2022-05-09 17:51:28.585930: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: NN_models/keras_model_step_1/assets


In [ ]:
# fill in the missing death day data using both clinic data and ct data
y_fill_test_predict = model.predict(X_fill_test)
for i in range(len(fill_test_idx)):
    if y_fill_test_predict[i] < 0:
        Outcome_Data[0][fill_test_idx[i]] = 0
    else:
        Outcome_Data[0][fill_test_idx[i]] = y_fill_test_predict[i]

np_outcome = np.array(Outcome_Data)
np_ctdata = np.array(CT_Data)
np_outcome = np_outcome.T
np_ctdata = np_ctdata.T
print(np_outcome.shape)
print(np_ctdata.shape)

out_csv = np.column_stack((np_outcome,np_ctdata))
out_csv = out_csv.astype(float)

np.savetxt("test_death_day_1.csv", out_csv, fmt='%f',delimiter=",")

(9223, 25)
(9223, 11)


/tmp/ipykernel_368642/407643235.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np_outcome = np.array(Outcome_Data)


# Step 2 - Use only ct data to predict death day
Note that we have only around 549 real samples with known death day for us to train the model

In this case, we use only CT data to predict death, we use the 549 samples with death day as training set, and predict the samples with missing death day

In [ ]:
# reread the data to clear variables
Clininc_Data,Outcome_Data,CT_Data = read_inputs()

In [ ]:
# prepare data for training, this case we use ct data only
X1 = np.array(CT_Data)
X1 = X1.T
X1 = X1.astype(float)
print(X1.shape)

X_fill_train_2 = X1[fill_train_idx,:]
print(X_fill_train_2.shape)

X_fill_test_2 = X1[fill_test_idx,:]
print(X_fill_test_2.shape)

y_fill_train_2 = np.array(Outcome_Data[0])
y_fill_train_2 = y_fill_train_2.astype(int)
y_fill_train_2 = np.asmatrix(y_fill_train_2)
y_fill_train_2 = y_fill_train_2.T
y_fill_train_2 = y_fill_train_2[fill_train_idx,:]
print(y_fill_train_2.shape)


(9223, 11)
(549, 11)
(8674, 11)
(549, 1)


In [ ]:
# train the model used for step 2
if re_train_2 == True:
    models_2 = []
    evas_2 = []
    counter = 0
    for train_index,test_index in KFold(n_split).split(X_fill_train_2):
        print("training fold "+str(counter)+" total "+str(n_split) +" folds")

        x_train,x_test = X_fill_train_2[train_index], X_fill_train_2[test_index]
        y_train,y_test = y_fill_train_2[train_index], y_fill_train_2[test_index]
        # define the keras model
        model_2 = Sequential()
        model_2.add(Dense(32, input_dim=11, use_bias=True))
        model_2.add(Dense(64, activation='relu',use_bias=True))
        model_2.add(Dense(128, activation='relu',use_bias=True))
        model_2.add(Dense(256, activation='relu',use_bias=True))
        model_2.add(Dense(512, activation='relu',use_bias=True))
        model_2.add(Dense(1))
        opt = keras.optimizers.Adam(learning_rate=0.0004)
        # compile the keras model
        model_2.compile(loss='MeanSquaredError', optimizer=opt, metrics=['MeanSquaredError'])
        # fit the keras model on the dataset
        model_2.fit(x_train, y_train, epochs=5000, batch_size=10,verbose=1)
        eva_2 = model_2.evaluate(x_test,y_test)

        print("fold "+str(counter)+" eva: "+str(eva_2[0]))
        counter = counter + 1

        models_2.append(model_2)
        evas_2.append(eva_2)
    for i in range(len(evas_2)):
        evas_2[i] = evas_2[i][0]
    # find the best evaluated models
    model_2 = models_2[evas_2.index(min(evas_2))]
else:
    model_2 = tf.keras.models.load_model("NN_models/keras_model_step_2", custom_objects=None, compile=True, options=None)

In [ ]:
# save/evaluate the model
from datetime import datetime
if re_train_2 == True:
    model_2.save("NN_models/keras_model_step_2")
score_2 = model_2.evaluate(X_fill_train_2, y_fill_train_2, verbose = 1) 
print(score_2)

# let's see how the model fits the original training set
err_sum = 0
y_fill_train_predict_2 = model_2.predict(X_fill_train_2)
for i in range((y_fill_train_predict_2.shape)[0]):
    err_sum = err_sum + (abs(y_fill_train_predict_2[i,0]-y_fill_train_2[i,0]))
print("average death day error across all 549 traing sample: "+str(err_sum/(y_fill_train_predict_2.shape)[0]))

18/18 [==============================] - 0s 602us/step - loss: 3.3247 - mean_squared_error: 3.3247
[3.324714183807373, 3.324714183807373]
average death day error across all 549 traing sample: 1.1271004911328926


In [ ]:
# predict and fill the missing death day field
y_fill_test_predict_2 = model_2.predict(X_fill_test_2)

for i in range(len(fill_test_idx)):
    if y_fill_test_predict_2[i] < 0:
        Outcome_Data[0][fill_test_idx[i]] = 0
    else:
        Outcome_Data[0][fill_test_idx[i]] = y_fill_test_predict_2[i]

np_outcome = np.array(Outcome_Data)
np_ctdata = np.array(CT_Data)
np_outcome = np_outcome.T
np_ctdata = np_ctdata.T
print(np_outcome.shape)
print(np_ctdata.shape)

out_csv = np.column_stack((np_outcome,np_ctdata))
out_csv = out_csv.astype(float)

np.savetxt("test_death_day_2.csv", out_csv, fmt='%f',delimiter=",")

(9223, 25)
(9223, 11)


/tmp/ipykernel_368642/3638360185.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np_outcome = np.array(Outcome_Data)
